In [145]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
import re
import string


In [146]:
data_fake=pd.read_csv("Fake.csv")
data_true=pd.read_csv("True.csv")

In [147]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [148]:
data_fake["class"]=0
data_true["class"]=1


In [149]:
data_true.shape

(21417, 5)

In [150]:
data_fake_mannual_testing=data_fake.tail()
for i in range(23480,23470,-1):
    data_fake.drop([i],axis=0,inplace=True)
data_true_mannual_testing=data_fake.tail()
for i in range(21416,21406,-1):
    data_true.drop([i],axis=0,inplace=True)
    

In [151]:
data_fake_mannual_testing["class"]=0
data_true_mannual_testing["class"]=1

C:\Users\hp\AppData\Local\Temp\ipykernel_14148\3158854425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_mannual_testing["class"]=0
C:\Users\hp\AppData\Local\Temp\ipykernel_14148\3158854425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_mannual_testing["class"]=1


In [152]:
data_fake_mannual_testing.head()

,title,text,subject,date,class
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [153]:
data_true_mannual_testing.head()

,title,text,subject,date,class
23466,Boston Brakes? How to Hack a New Car With Your...,21st Century Wire says For those who still ref...,Middle-east,"January 22, 2016",1
23467,Oregon Governor Says Feds ‘Must Act’ Against P...,"21st Century Wire says So far, after nearly 20...",Middle-east,"January 21, 2016",1
23468,Ron Paul on Burns Oregon Standoff and Jury Nul...,21st Century Wire says If you ve been followin...,Middle-east,"January 21, 2016",1
23469,BOILER ROOM: As the Frogs Slowly Boil – EP #40,Tune in to the Alternate Current Radio Network...,Middle-east,"January 20, 2016",1
23470,Arizona Rancher Protesting in Oregon is Target...,RTOne of the most visible members of the armed...,Middle-east,"January 20, 2016",1


In [154]:
data_merge=pd.concat([data_fake,data_true],axis=0)

In [155]:
data_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [156]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [157]:
data=data_merge.drop(["title","subject","date"],axis=1)

In [158]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [159]:
data=data.sample(frac=1)

In [160]:
data.head()

,text,class
1517,(Reuters) - Four U.S. senators have asked the ...,1
6781,"BATON ROUGE, La. (Reuters) - Republican John N...",1
10593,Joy Reid is known for her racist slant on the ...,0
6140,Like a stubborn cold that just doesn t seem to...,0
23315,Tune in to the Alternate Current Radio Network...,0


In [161]:
data.reset_index(inplace=True)
data.drop(["index"],axis=1,inplace=True)

In [162]:
data.columns

Index(['text', 'class'], dtype='object')

In [163]:
def wordopt(text):
    text=text.lower()
    text=re.sub('\[.*?\]',"",text)
    text=re.sub("\\W"," ",text)
    text=re.sub("https?:://\S+|www\.\S+","",text)
    text=re.sub("<.*?>+","",text)
    text=re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text=re.sub("\n","",text)
    text=re.sub("\w*\d\w*","",text)
    return text




In [164]:
data["text"]=data["text"].apply(wordopt)

In [165]:
x=data["text"]
y=data["class"]

In [166]:
x.shape

(44878,)

In [167]:
y

0        1
1        1
2        0
3        0
4        0
        ..
44873    0
44874    0
44875    0
44876    1
44877    1
Name: class, Length: 44878, dtype: int64

In [168]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25)

In [169]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(33658,)
(33658,)
(11220,)
(11220,)


In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization=TfidfVectorizer()
xv_train=vectorization.fit_transform(xtrain)
xv_test=vectorization.transform(xtest)


In [206]:
dump(vectorization, 'vectorizer.jb')

['vectorizer.jb']

In [171]:
print(xv_train.shape)
print(xv_test.shape)


(33658, 94925)
(11220, 94925)


In [172]:
from sklearn.linear_model import LogisticRegression 
lr=LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [205]:
from joblib import dump, load
dump(lr, 'LogisticRegression.jb') 


['LogisticRegression.jb']

In [173]:
print(xv_test.shape)
print(xtest.shape)
print(xtrain.shape) 
print(xv_train.shape)

(11220, 94925)
(11220,)
(33658,)
(33658, 94925)


In [174]:
predict_lr=lr.predict(xv_test)

In [175]:
lr.score(xv_test,ytest)

0.9870766488413547

In [176]:
print(classification_report(ytest,predict_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5830
           1       0.98      0.99      0.99      5390

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [177]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(xv_train,ytrain)

DecisionTreeClassifier()

In [178]:

pred_dt=dt.predict(xv_test)

In [179]:

dt.score(xv_test,ytest)
print(classification_report(ytest,pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5830
           1       1.00      1.00      1.00      5390

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [204]:
from joblib import dump, load
dump(dt, 'DecisionTree.jb') 


['DecisionTree.jb']

In [180]:
from sklearn.ensemble import GradientBoostingClassifier

In [181]:
gb=GradientBoostingClassifier()
gb.fit(xv_train,ytrain)

KeyboardInterrupt: 

In [ ]:

pred_gb=gb.predict(xv_test)
gb.score(xv_test,ytest)
print(classification_report(ytest,pred_gb))

In [185]:
from sklearn.ensemble import RandomForestClassifier

In [186]:
rf=RandomForestClassifier(random_state=0)
rf.fit(xv_train,ytrain)
pred_rf=dt.predict(xv_test)
rf.score(xv_test,ytest)
print(classification_report(ytest,pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5830
           1       1.00      1.00      1.00      5390

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [203]:
from joblib import dump, load
dump(rf, 'randomforest.jb') 


['randomforest.jb']

In [193]:
def output_label(n):

    if n == 0:

        return "Fake News"

    elif n == 1:

        return "Not A Fake News"


In [196]:
def manual_testing(news):

    testing_news={"text":[news]}

    new_def_test = pd.DataFrame(testing_news)

    new_def_test["text"] = new_def_test["text"].apply(wordopt)

    new_x_test =new_def_test["text"]

    new_xv_test = vectorization.transform(new_x_test)

    predict_lr= lr.predict(new_xv_test)

    pred_dt=dt.predict(new_xv_test)

    pred_rf =rf.predict(new_xv_test)
    return print("\n LR Prediction:{}  \n DT Prediction:{}  \n RFC Prediction:{} \n ".format(output_label(predict_lr[0]),output_label(pred_dt[0]),output_label(pred_rf[0])))

In [207]:
news=str(input())
manual_testing(news)


 LR Prediction:Not A Fake News  
 DT Prediction:Not A Fake News  
 RFC Prediction:Not A Fake News 
 


In [ ]:
from joblib import dump, load
dump(model, 'CNN.model') 
